Ideas 1:
1. first we need to convert our sparse matrix to dense matrix where each row is an example by keeping only the diagonals.
2. Change in Shapes:
   1. for 1D models: 
      1. the shape will change from (nxn) to (nx3) since each cell has 3 terms (i.e. 2 neighbors and the cell itself). 
      2. Formulation:
         1.  b, c1, c2 x x1 > b1
         2. c1, c2, c3 x x2 > b2
         3. c2, c3, c4 x x3 > b3
         4. c3, c4, b  x x4 > b4
   2. for 2D models: 
      1. the shape will change from (nxn) to (nx5) since each cell has 5 terms (i.e. 4 neighbors and the cell itself).  
   3. for 3D models:
      1. the shape will change from (nxn) to (nx7) since each cell has 7 terms (i.e. 6 neighbors and the cell itself).

Idea 2: Train neural network to calc A^-1 then use that to get x: A^-1.b = x

In [10]:
import openresim as ors
import tensorflow as tf

In [11]:
def create_model():
    grid = ors.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = ors.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = ors.models.Model(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model

model = create_model()

In [19]:
[print(f"cell {k}: {dict(v[0])}") for k, v in model.get_cells_eq().items()]

cell 1: {p1: 85.2012000000000, p2: -28.4004000000000}
cell 2: {p1: 28.4004000000000, p3: 28.4004000000000, p2: -56.8008000000000}
cell 3: {p2: 28.4004000000000, p4: 28.4004000000000, p3: -56.8008000000000}
cell 4: {p3: 28.4004000000000, p4: -28.4004000000000}


[None, None, None, None]

In [20]:
model.get_matrices_vectorized()

(array([[-85.2012,  28.4004,   0.    ,   0.    ],
        [ 28.4004, -56.8008,  28.4004,   0.    ],
        [  0.    ,  28.4004, -56.8008,  28.4004],
        [  0.    ,   0.    ,  28.4004, -28.4004]]),
 array([[-227203.2],
        [      0. ],
        [      0. ],
        [    600. ]]))

In [28]:
list(model.pressures[0])

[4000.0, nan, nan, nan, nan, nan]

In [23]:
model.run(
        nsteps=1,
        sparse=True,
        threading=True,
        vectorize=True,
        check_MB=True,
        print_arrays=True,
        isolver="cgs")

[info] Simulation run started: 1 timesteps.


100%|██████████| 1/1 [00:00<00:00,  5.94steps/s]

step: 0
[[ 85.2012 -28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [-85.2012  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [170.4024 -56.8008   0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
[[ 227203.2 -227203.2  454406.4]
 [      0.        0.        0. ]
 [      0.        0.        0. ]
 [    600.      600.        0. ]]

[info] Simulation run of 1 steps finished in 0.17 seconds.

[info] Material Balance Error: -1.4909971923771081e-07.


In [27]:
list(model.pressures[1])

[4000.0,
 3989.4367684988433,
 3968.310305489209,
 3947.1838424812486,
 3926.057379473549,
 nan]